# Data Preparation

In [291]:
# Load nyc_geo JSON
import json
import pandas as pd
from pandas.io.json import json_normalize

file = open('data/nyc_geo.json')
data = json.load(file)

### Task
Parse the json **nyc_geo.json** into the dataframe with the following columns:
- Borough
- Neighborhood
- Latitude
- Longitude

In [292]:
data.keys()

dict_keys(['type', 'totalFeatures', 'features', 'crs', 'bbox'])

In [293]:
df = pd.json_normalize(data['features'])
df.head()

,type,id,geometry_name,geometry.type,geometry.coordinates,properties.name,properties.stacked,properties.annoline1,properties.annoline2,properties.annoline3,properties.annoangle,properties.borough,properties.bbox
0,Feature,nyu_2451_34572.1,geom,Point,"[-73.84720052054902, 40.89470517661]",Wakefield,1,Wakefield,None,None,0.0,Bronx,"[-73.84720052054902, 40.89470517661, -73.84720..."
1,Feature,nyu_2451_34572.2,geom,Point,"[-73.82993910812398, 40.87429419303012]",Co-op City,2,Co-op,City,None,0.0,Bronx,"[-73.82993910812398, 40.87429419303012, -73.82..."
2,Feature,nyu_2451_34572.3,geom,Point,"[-73.82780644716412, 40.887555677350775]",Eastchester,1,Eastchester,None,None,0.0,Bronx,"[-73.82780644716412, 40.887555677350775, -73.8..."
3,Feature,nyu_2451_34572.4,geom,Point,"[-73.90564259591682, 40.89543742690383]",Fieldston,1,Fieldston,None,None,0.0,Bronx,"[-73.90564259591682, 40.89543742690383, -73.90..."
4,Feature,nyu_2451_34572.5,geom,Point,"[-73.9125854610857, 40.890834493891305]",Riverdale,1,Riverdale,None,None,0.0,Bronx,"[-73.9125854610857, 40.890834493891305, -73.91..."


In [294]:
# Separate coordinates into longitude and latitude columns
longitude = []
latitude = []

for row in df['geometry.coordinates']:
    longitude.append(row[0])
    latitude.append(row[1])

df['longitude'] = longitude
df['latitude'] = latitude

In [295]:
# Tidy Data
nyc_df = df[['id','properties.borough', 'properties.name', 'longitude', 'latitude']]
nyc_df.columns = ['id', 'borough', 'neighborhood', 'longitude', 'latitude']
nyc_df

,id,borough,neighborhood,longitude,latitude
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834
...,...,...,...,...,...
301,nyu_2451_34572.302,Manhattan,Hudson Yards,-74.000111,40.756658
302,nyu_2451_34572.303,Queens,Hammels,-73.805530,40.587338
303,nyu_2451_34572.304,Queens,Bayswater,-73.765968,40.611322
304,nyu_2451_34572.305,Queens,Queensbridge,-73.945631,40.756091


In [296]:
 # csv = nyc_df.to_csv('nyc_geo_data_cleaned.csv')

### Task
Use different data sources and APIs to collect information about the neigborhoods that can be used for segmentation.

In [297]:
populations = pd.read_csv('data/nyc_populations.csv')
populations = populations[['neighborhood', 'population']]
populations.head()

,neighborhood,population
0,Wakefield,21242.0
1,Co-op City,43752.0
2,Eastchester,11506.0
3,Fieldston,9287.0
4,Riverdale,9287.0


In [298]:
nyc_df = pd.merge(nyc_df, populations, on='neighborhood')
nyc_df = nyc_df.drop_duplicates()

In [299]:
nyc_df = nyc_df.drop(index=[120,121, 147, 148, 185, 186])

In [300]:
nyc_df.shape

(306, 6)

In [301]:
nyc_df.head()

,id,borough,neighborhood,longitude,latitude,population
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705,21242.0
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294,43752.0
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556,11506.0
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437,9287.0
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834,9287.0


In [317]:
resturants = pd.read_csv('data/resturants_data.csv')
resturants = resturants.iloc[:, 1:]
resturants.head()

,id,number_of_restaurants,average_review_count,average_restaurant_rating,average_price_level
0,nyu_2451_34572.1,13,169,4.0,1
1,nyu_2451_34572.2,12,771,4.0,2
2,nyu_2451_34572.3,19,532,4.1,1
3,nyu_2451_34572.4,9,297,4.4,1
4,nyu_2451_34572.5,14,319,4.0,2


In [303]:
nyc_df = pd.merge(nyc_df, resturants, on='id')
nyc_df = nyc_df.drop_duplicates()

In [304]:
nyc_df.head()

,id,borough,neighborhood,longitude,latitude,population,number_of_restaurants,average_review_count,average_restaurant_rating,average_price_level
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705,21242.0,13,169,4.0,1
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294,43752.0,12,771,4.0,2
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556,11506.0,19,532,4.1,1
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437,9287.0,9,297,4.4,1
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834,9287.0,14,319,4.0,2


In [312]:
yelp_categories = pd.read_csv('data/yelp_restaurant_categories.csv')
yelp_categories = yelp_categories[['id', 'yelp_category']]
yelp_categories.head()

,id,yelp_category
0,nyu_2451_34572.1,caribbean
1,nyu_2451_34572.2,delis
2,nyu_2451_34572.3,caribbean
3,nyu_2451_34572.4,N
4,nyu_2451_34572.5,burgers


In [313]:
nyc_df = pd.merge(nyc_df, yelp_categories, on='id')

In [314]:
nyc_df.head()

,id,borough,neighborhood,longitude,latitude,population,number_of_restaurants,average_review_count,average_restaurant_rating,average_price_level,yelp_category
0,nyu_2451_34572.1,Bronx,Wakefield,-73.847201,40.894705,21242.0,13,169,4.0,1,caribbean
1,nyu_2451_34572.2,Bronx,Co-op City,-73.829939,40.874294,43752.0,12,771,4.0,2,delis
2,nyu_2451_34572.3,Bronx,Eastchester,-73.827806,40.887556,11506.0,19,532,4.1,1,caribbean
3,nyu_2451_34572.4,Bronx,Fieldston,-73.905643,40.895437,9287.0,9,297,4.4,1,N
4,nyu_2451_34572.5,Bronx,Riverdale,-73.912585,40.890834,9287.0,14,319,4.0,2,burgers


In [315]:
# Check for missing data
total = nyc_df.isnull().sum().sort_values(ascending=False)
percent = (nyc_df.isnull().sum()/nyc_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

,Total,Percent
id,0,0.0
borough,0,0.0
neighborhood,0,0.0
longitude,0,0.0
latitude,0,0.0


In [316]:
# save_file = nyc_df.to_csv('main_dataset.csv')

### Task

Visualize the neigborhoods in the graph.